In [20]:
#Import packages
#---------------------------------------
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
import warnings
import random
import arviz as az
import pymc as pm
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#Import your modules
#---------------------------------------
import admin_functions as adfn
import cell_decomp_func as cdfn


# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
%load_ext autoreload
sys.version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]'

In [26]:
gamma_list = np.geomspace(5,100,20).astype(int)
gamma_list[0]=0
gamma_list[-1]=99

array([ 0,  5,  6,  8,  9, 10, 12, 15, 17, 20, 24, 28, 33, 38, 45, 53, 62,
       72, 85, 99])

In [5]:
# Run alpha noise

#Define parameters of simulated data
n_clusts = 5
n_genes = 800
n_cells = 100
rate_range = 0,40 #max and min of uniform distribution for generating rates
mode = 'gamma' #CHANGE!
noise_type = 'gaussian' #CHANGE!

for x in gamma_list: #CHANGE!
    #Simulate spot data from simulated gene expression
    spot_sim = cdfn.simulate_cell_mix(n_clusts, n_cells, n_genes).simulate_gene_exp(rate_range)
    n_spots = spot_sim.__dict__['n_spots']
    spots = spot_sim.__dict__['spots']
    ref_exp = spot_sim.__dict__['mean_exps']
    prop_vec = spot_sim.__dict__['prop_vec']
    spots = cdfn.add_noise(spots, per=None, a_std=None, g_std=x, e_std=None, noise_type=noise_type) #CHANGE!

    #Run basic model
    basic_model = cdfn.basic_pymc(n_clusts, n_spots, n_genes, ref_exp)
    with basic_model: basic_data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)
    
    #Run noise model
    noise_model = cdfn.gamma_pymc(n_clusts, n_spots, n_genes, ref_exp, noise_type) #CHANGE!
    with noise_model: noise_data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)

    basic_post, basic_r2 = cdfn.model_stats(basic_data, prop_vec)
    noise_post, noise_r2 = cdfn.model_stats(noise_data, prop_vec)
    if len(str(x))<2:pref = '0' + str(x)
    else:pref= str(x)
    np.save(s_data + 'spatial_transcriptomics/RCTD-test-basic-' + noise_type + '-model_'+ mode + '-' + pref + '.npy', np.array([prop_vec, basic_post, basic_r2], dtype=object))
    np.save(s_data + 'spatial_transcriptomics/RCTD-test-noise-' + noise_type + '-model_' + mode + '-' + pref + '.npy', np.array([prop_vec, noise_post, noise_r2], dtype=object))
    print(x)

05 5
06 6
08 8
10 10
13 13
17 17
23 23
29 29
38 38
50 50
